In [9]:
import pertpy as pt
import scanpy as sc
import anndata as ad
import os
import matplotlib.pyplot as plt    
import time

import pandas as pd

data_dir = "/home/ubuntu/frameshift-1/data/h5ad"
augur_out = "/home/ubuntu/frameshift-1/data/augur_out"

In [4]:
parquet_path = "/home/ubuntu/frameshift-1/data/metadata/obs_metadata.parquet"
obs_meta = pd.read_parquet(path=parquet_path, engine="pyarrow")
obs_meta

,plate,BARCODE_SUB_LIB_ID,sample,gene_count,tscp_count,mread_count,drugname_drugconc,drug,cell_line,sublibrary,BARCODE,pcnt_mito,S_score,G2M_score,phase,pass_filter,cell_name
0,plate10,01_001_001-lib_1681,smp_2359,1379,2172,2559,"[('Bestatin (hydrochloride)', 0.05, 'uM')]",Bestatin (hydrochloride),CVCL_1478,lib_1681,01_001_001,0.029926,-0.229665,-0.190110,G1,full,NCI-H1573
1,plate10,01_002_149-lib_1681,smp_2359,975,1256,1470,"[('Bestatin (hydrochloride)', 0.05, 'uM')]",Bestatin (hydrochloride),CVCL_0459,lib_1681,01_002_149,0.026274,-0.167578,-0.132784,G1,full,NCI-H460
2,plate10,01_003_052-lib_1681,smp_2359,865,1239,1446,"[('Bestatin (hydrochloride)', 0.05, 'uM')]",Bestatin (hydrochloride),CVCL_C466,lib_1681,01_003_052,0.033898,-0.200957,-0.161538,G1,full,hTERT-HPNE
3,plate10,01_003_090-lib_1681,smp_2359,393,484,559,"[('Bestatin (hydrochloride)', 0.05, 'uM')]",Bestatin (hydrochloride),CVCL_1724,lib_1681,01_003_090,0.037190,-0.052746,-0.076190,G1,minimal,SW48
4,plate10,01_003_093-lib_1681,smp_2359,2657,5325,6269,"[('Bestatin (hydrochloride)', 0.05, 'uM')]",Bestatin (hydrochloride),CVCL_1285,lib_1681,01_003_093,0.017465,-0.636364,-0.614103,G1,full,HOP62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100648785,plate9,96_191_161-lib_2608,smp_2358,1763,2475,2852,"[('DMSO_TF', 0.0, 'uM')]",DMSO_TF,CVCL_0293,lib_2608,96_191_161,0.075960,-0.129528,0.146154,G2M,full,HEC-1-A
100648786,plate9,96_191_173-lib_2608,smp_2358,656,800,942,"[('DMSO_TF', 0.0, 'uM')]",DMSO_TF,CVCL_0504,lib_2608,96_191_173,0.041250,-0.047961,0.024725,G2M,full,RKO
100648787,plate9,96_192_081-lib_2608,smp_2358,2872,4992,5729,"[('DMSO_TF', 0.0, 'uM')]",DMSO_TF,CVCL_0320,lib_2608,96_192_081,0.064704,-0.039417,0.411355,G2M,full,HT-29
100648788,plate9,96_192_166-lib_2608,smp_2358,1438,2139,2471,"[('DMSO_TF', 0.0, 'uM')]",DMSO_TF,CVCL_1119,lib_2608,96_192_166,0.044881,-0.148553,-0.022161,G1,full,CFPAC-1


In [6]:

	# drugs = adata.obs

	# # Filter the data
	# filtered_adata = adata[adata.obs["cell_type"] == "Tumor"]
	
	# # Save the filtered data
	# filtered_adata.write_h5ad(os.path.join(data_dir, f"{plate}_filtered.h5ad"))
	
	# Optionally, you can also save the filtered data in a different format
	# filtered_adata.write_parquet(os.path.join(data_dir, f"{plate}_filtered.parquet"))

Processing plate: plate10


IndentationError: unindent does not match any outer indentation level (<string>, line 6)

In [50]:
control = "[('DMSO_TF', 0.0, 'uM')]"
for plate in obs_meta["plate"].unique():
	print(f"Processing plate: {plate}")
	# Load the data
	adata = ad.read_h5ad(f"{data_dir}/{plate}_filt_Vevo_Tahoe100M_WServicesFrom_ParseGigalab.h5ad", backed="r")
	drugs = adata.obs["drugname_drugconc"].unique()
	drugs = drugs[drugs != control]
	for drug in drugs:
		ag_rfc = pt.tl.Augur("random_forest_classifier")
		print(drug)
		drug_data = adata[adata.obs["drugname_drugconc"].isin([control, drug])].to_memory()
		drug_data.obs['condition'] = drug_data.obs['drug']
		drug_data.obs["condition"].replace({control: "ctrl", drug: "stim"}, inplace=True)
		sc.pp.normalize_total(drug_data, target_sum=1e6)
		sc.pp.log1p(drug_data)
		sc.pp.highly_variable_genes(drug_data, flavor="seurat", n_top_genes=2000)
		loaded_data = ag_rfc.load(drug_data,label_col="condition", cell_type_col="cell_name")
		v_adata, v_results = ag_rfc.predict(loaded_data, subsample_size=20, n_threads=20, select_variance_features=False, span=1)

		df = v_results["summary_metrics"]
		df["drug"] = drug
		filename = f"{augur_out}/scores/{plate}__{drug}.csv"
		df.to_csv(filename, index=False)
		print("saved!")
		print("saving important features...")
		print(v_results["feature_importances"])
		df = v_results["feature_importances"]
		df["drug"] = drug
		filename = f"{augur_out}/importance/{plate}__{drug}.csv"
		df.to_csv(filename, index=False)
		break
	break

Processing plate: plate10
[('Bestatin (hydrochloride)', 0.05, 'uM')]


/tmp/ipykernel_209570/2887447007.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  drug_data.obs["condition"].replace({control: "ctrl", drug: "stim"}, inplace=True)


NameError: name 'ag_rfc' is not defined